In [62]:
import torch

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

device = 'cuda' if torch.cuda.is_available() else 'cpu'

embedder = SentenceTransformer('all-MiniLM-L6-v2', device=device)
dutch_embedder = SentenceTransformer("jegormeister/robbert-v2-dutch-base-mqa-finetuned", device=device)
# dutch_embedder = SentenceTransformer('GroNLP/gpt2-medium-dutch-embeddings', device=device)


In [59]:
# Corpus with example documents
dutch_corpus_highschool = [
    "De vrijheid onder de Fransen valt tegen. Frankrijk trekt steeds meer macht naar zich toe. Keizer Napoleon benoemt eerst zijn broer Lodewijk Napoleon tot koning en lijft daarna Nederland bij Frankrijk in. In 1813 verliest Napoleon de strijd van zijn tegenstanders en wordt Nederland weer onafhankelijk.",
    
    "Napoleon speelt een hoofdrol in de Europese geschiedenis. In 1799 pleegt hij een militaire staatsgreep in Frankrijk. Hij streeft een groot rijk na en voert verschillende oorlogen. Als generaal leidt hij zijn troepen in de veldtochten tegen de keizer van Oostenrijk, de Russische tsaar, de sultan van het Ottomaanse Rijk en de Engelse koning. In 1804 kroont Napoleon zichzelf tot keizer. Hij heerst vanaf 1806 over bijna heel Europa, dat hij bestuurt als ‘verlicht despoot’: een alleenheerser die zich laat leiden door de ideeën van de Verlichting zolang het hem uitkomt. Van Nederland maakt hij een Franse vazalstaat.",
    
    "Met hulp van Franse troepen brengen de patriotten in 1795 de regering van de oude Republiek ten val en richten in plaats daarvan de Bataafse Republiek op. Tot 1806 blijft de Bataafse Republiek formeel onafhankelijk van Frankrijk, maar op den duur gebeurt er maar weinig zonder goedkeuring van de Fransen. In 1806 benoemt Napoleon zijn broer Lodewijk Napoleon tot koning van Holland. Voor het eerst is Nederland een koninkrijk, dat de basis vormt voor de latere monarchie."
    
    ""
]

dutch_queries = [
    "Napoleon was goed voor Nederland",
    "Napoleon was slechte voor Nederland"
]

corpus = dutch_corpus_highschool
queries = dutch_queries

In [60]:
k=5
top_k = min(k, len(corpus))

corpus_embeddings = embedder.encode_document(corpus, convert_to_tensor=True)

for query in queries:
    query_embedding = embedder.encode_query(query, convert_to_tensor=True)

    # We use cosine-similarity and torch.topk to find the highest 5 scores
    similarity_scores = embedder.similarity(query_embedding, corpus_embeddings)[0]
    scores, indices = torch.topk(similarity_scores, k=top_k)

    print("\nQuery:", query)
    print(f"Top {k} most similar sentences in corpus:")

    for score, idx in zip(scores, indices):
        print(f"(Score: {score:.4f})", corpus[idx])

    """
    # Alternatively, we can also use util.semantic_search to perform cosine similarty + topk
    hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=5)
    hits = hits[0]      #Get the hits for the first query
    for hit in hits:
        print(corpus[hit['corpus_id']], "(Score: {:.4f})".format(hit['score']))
    """


Query: Napoleon was goed voor Nederland
Top 5 most similar sentences in corpus:
(Score: 0.6920) Napoleon komt in 1815 nog even aan de macht in Frankrijk, maar lijdt zijn definitieve nederlaag bij Waterloo.
(Score: 0.6902) In 1813 verliest Napoleon de strijd van zijn tegenstanders en wordt Nederland weer onafhankelijk.
(Score: 0.6822) Napoleon speelt een hoofdrol in de Europese geschiedenis.
(Score: 0.6817) In 1810 zet Napoleon zijn broer af en lijft hij Nederland in bij het Franse Keizerrijk.
(Score: 0.6800) In 1806 benoemt Napoleon zijn broer Lodewijk Napoleon tot koning van Holland.

Query: Napoleon was slechte voor Nederland
Top 5 most similar sentences in corpus:
(Score: 0.7007) Napoleon speelt een hoofdrol in de Europese geschiedenis.
(Score: 0.6930) Napoleon komt in 1815 nog even aan de macht in Frankrijk, maar lijdt zijn definitieve nederlaag bij Waterloo.
(Score: 0.6873) In 1810 zet Napoleon zijn broer af en lijft hij Nederland in bij het Franse Keizerrijk.
(Score: 0.6864) In 

In [61]:
k=5
top_k = min(k, len(corpus))

corpus_embeddings = dutch_embedder.encode_document(corpus, convert_to_tensor=True)

for query in queries:
    query_embedding = dutch_embedder.encode_query(query, convert_to_tensor=True)

    # We use cosine-similarity and torch.topk to find the highest 5 scores
    similarity_scores = dutch_embedder.similarity(query_embedding, corpus_embeddings)[0]
    scores, indices = torch.topk(similarity_scores, k=top_k)

    print("\nQuery:", query)
    print(f"Top {k} most similar sentences in corpus:")

    for score, idx in zip(scores, indices):
        print(f"(Score: {score:.4f})", corpus[idx])

    """
    # Alternatively, we can also use util.semantic_search to perform cosine similarty + topk
    hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=5)
    hits = hits[0]      #Get the hits for the first query
    for hit in hits:
        print(corpus[hit['corpus_id']], "(Score: {:.4f})".format(hit['score']))
    """


Query: Napoleon was goed voor Nederland
Top 5 most similar sentences in corpus:
(Score: 0.8266) Napoleon speelt een hoofdrol in de Europese geschiedenis.
(Score: 0.7048) Napoleon komt in 1815 nog even aan de macht in Frankrijk, maar lijdt zijn definitieve nederlaag bij Waterloo.
(Score: 0.6269) Tegenstanders hebben als kritiek dat de Code Napoléon geen rekening houdt met de zo verschillende lokale gewoonten en afspraken.
(Score: 0.5715) Zo blijft de Code Napoléon bestaan, net als veel andere moderniseringen.
(Score: 0.5522) De vrijheid onder de Fransen valt tegen.

Query: Napoleon was slechte voor Nederland
Top 5 most similar sentences in corpus:
(Score: 0.8359) Napoleon speelt een hoofdrol in de Europese geschiedenis.
(Score: 0.6992) Napoleon komt in 1815 nog even aan de macht in Frankrijk, maar lijdt zijn definitieve nederlaag bij Waterloo.
(Score: 0.6356) Tegenstanders hebben als kritiek dat de Code Napoléon geen rekening houdt met de zo verschillende lokale gewoonten en afspraken.

In [74]:
### MMR 

def calculate_mmr(query_embedding, document_embeddings, lambda_param=0.7, top_k=5):
    """
    Calculates the Maximal Marginal Relevance (MMR) for document reranking.
    Args:
        query_embedding (np.array): The embedding of the query.
        document_embeddings (list of np.array): List of embeddings for the documents.
        lambda_param (float): Balances relevance and diversity (0 <= lambda_param <= 1).
        top_k (int): Number of documents to return.
    Returns:
        list: Indices of the top-k documents selected using MMR.
    """
    # Calculate cosine similarity of documents with the query
    query_similarities = cosine_similarity([query_embedding], document_embeddings)[0]
    
    # Initialize variables
    selected_indices = []
    best_scores = []
    
    remaining_indices = list(range(len(document_embeddings)))
    for _ in range(top_k):
        mmr_scores = []
        
        for i in remaining_indices:
            # Calculate diversity term
            diversity_score = max(
                cosine_similarity([document_embeddings[i]], [document_embeddings[j]])[0][0]
                for j in selected_indices
            ) if selected_indices else 0
            
            # MMR formula
            mmr_score = lambda_param * query_similarities[i] - (1 - lambda_param) * diversity_score
            mmr_scores.append((i, mmr_score))
        # Select document with highest MMR score
        best_doc = max(mmr_scores, key=lambda x: x[1])
        selected_indices.append(best_doc[0])
        remaining_indices.remove(best_doc[0])
        best_scores.append(best_doc[1])
    return selected_indices, best_scores

In [76]:
corpus_embeddings = dutch_embedder.encode_document(corpus)
k=5
for query in queries:
    query_embedding = dutch_embedder.encode_query(query)
    # print(query_embedding.shape)
    # print(corpus_embeddings.shape)
    indices,scores = calculate_mmr(query_embedding,corpus_embeddings,top_k=k) 
    print("\nQuery:", query)
    print(f"Top {k} most similar sentences in corpus:")
    for idx, score in zip(indices, scores):
        print(f"(Score: {score:.4f})", corpus[idx])
        


Query: Napoleon was goed voor Nederland
Top 5 most similar sentences in corpus:
(Score: 0.5786) Napoleon speelt een hoofdrol in de Europese geschiedenis.
(Score: 0.2715) Napoleon komt in 1815 nog even aan de macht in Frankrijk, maar lijdt zijn definitieve nederlaag bij Waterloo.
(Score: 0.2487) Tegenstanders hebben als kritiek dat de Code Napoléon geen rekening houdt met de zo verschillende lokale gewoonten en afspraken.
(Score: 0.2132) Van Nederland maakt hij een Franse vazalstaat.
(Score: 0.1951)  Voor het eerst is Nederland een koninkrijk, dat de basis vormt voor de latere monarchie.

Query: Napoleon was slechte voor Nederland
Top 5 most similar sentences in corpus:
(Score: 0.5851) Napoleon speelt een hoofdrol in de Europese geschiedenis.
(Score: 0.2676) Napoleon komt in 1815 nog even aan de macht in Frankrijk, maar lijdt zijn definitieve nederlaag bij Waterloo.
(Score: 0.2548) Tegenstanders hebben als kritiek dat de Code Napoléon geen rekening houdt met de zo verschillende lokale 